In [168]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [39]:
dataset_path = '../DATASET/INITIAL_DATASET'
train_path = os.path.join(dataset_path, 'train.csv')
test_path = os.path.join(dataset_path, 'test.csv')

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [206]:
train_df['source'] = 'train'
test_df['source'] = 'test'

full_df = pd.concat([train_df, test_df], ignore_index=True)

In [41]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [42]:
test_df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [222]:
df = full_df.copy()
le = LabelEncoder()
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')

In [223]:
df['Embarked'] = le.fit_transform(df['Embarked'])
df['Embarked'] = df['Embarked'].replace(3, np.nan)
ohetransform = ohe.fit_transform(df[['Sex']])
df = pd.concat([df, ohetransform], axis=1).drop(columns=['Sex'])


In [224]:
features = ['Pclass', 'Age', 'Fare', 'SibSp', 'Parch', 'Embarked', 'Sex_male', 'Sex_female']
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df[features]), columns=features)

In [225]:
df_imputed['Embarked'] = df_imputed['Embarked'].round().astype(int)
df_imputed['Embarked'] = le.inverse_transform(df_imputed['Embarked'])

In [226]:
full_df['Embarked'] = full_df['Embarked'].fillna(df_imputed['Embarked'])

In [229]:
full_df = pd.concat([full_df, ohetransform], axis=1).drop(columns=['Sex'])

In [271]:
full_df.dtypes

PassengerId      int64
Survived       float64
Pclass           int64
Name            object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked         int64
source          object
Sex_female     float64
Sex_male       float64
dtype: object